<a href="https://colab.research.google.com/github/Igwe-Ugo/Facial_recognition_system/blob/main/face_recog_resnet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Lambda, GlobalAveragePooling2D
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam

In [ ]:
# 1. Use TensorFlow's mixed precision
tf.keras.mixed_precision.set_global_policy('mixed_float16')

In [ ]:
# Define the dataset paths
data_path = 'drive/MyDrive/face_data'
train_path = os.path.join(data_path, 'train')
val_path = os.path.join(data_path, 'val')

# Create the directories if they don't exist
os.makedirs(train_path, exist_ok=True)
os.makedirs(val_path, exist_ok=True)

In [ ]:
def load_and_preprocess_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [224, 224])
    img = tf.keras.applications.resnet50.preprocess_input(img)
    return tf.cast(img, tf.float16)  # Cast to float16

In [ ]:
def create_triplet_dataset(directory):
    image_paths = []
    labels = []
    for person_dir in os.listdir(directory):
        person_path = os.path.join(directory, person_dir)
        if os.path.isdir(person_path):
            for image_name in os.listdir(person_path):
                image_paths.append(os.path.join(person_path, image_name))
                labels.append(person_dir)

    def generate_triplets():
        while True:
            anchor_label = random.choice(labels)
            anchor_image = random.choice([img for img, lbl in zip(image_paths, labels) if lbl == anchor_label])
            positive_image = random.choice([img for img, lbl in zip(image_paths, labels) if lbl == anchor_label and img != anchor_image])
            negative_label = random.choice([lbl for lbl in labels if lbl != anchor_label])
            negative_image = random.choice([img for img, lbl in zip(image_paths, labels) if lbl == negative_label])

            yield (load_and_preprocess_image(anchor_image),
                   load_and_preprocess_image(positive_image),
                   load_and_preprocess_image(negative_image))

    return tf.data.Dataset.from_generator(
        generate_triplets,
        output_signature=(
            tf.TensorSpec(shape=(224, 224, 3), dtype=tf.float32),
            tf.TensorSpec(shape=(224, 224, 3), dtype=tf.float32),
            tf.TensorSpec(shape=(224, 224, 3), dtype=tf.float32)
        )
    )

In [ ]:
def create_base_network(input_shape):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    return Model(inputs=base_model.input, outputs=x)

In [ ]:
@tf.function
def triplet_loss(y_true, y_pred, alpha = 0.2):
    anchor, positive, negative = y_pred[:,:128], y_pred[:,128:256], y_pred[:,256:]

    pos_dist = tf.reduce_sum(tf.square(anchor - positive), axis=-1)
    neg_dist = tf.reduce_sum(tf.square(anchor - negative), axis=-1)

    basic_loss = pos_dist - neg_dist + alpha
    loss = tf.maximum(basic_loss, 0.0)
    return tf.reduce_mean(loss)

In [ ]:
def create_model(input_shape):
    base_network = create_base_network(input_shape)

    input_anchor = Input(shape=input_shape, name='input_1')
    input_positive = Input(shape=input_shape, name='input_2')
    input_negative = Input(shape=input_shape, name='input_3')

    embedding_anchor = base_network(input_anchor)
    embedding_positive = base_network(input_positive)
    embedding_negative = base_network(input_negative)

    output = Lambda(lambda x: tf.concat(x, axis=-1))([embedding_anchor, embedding_positive, embedding_negative])

    model = Model(inputs=[input_anchor, input_positive, input_negative], outputs=output)

    optimizer = Adam(learning_rate=0.0001)
    optimizer = tf.keras.mixed_precision.LossScaleOptimizer(optimizer)

    model.compile(loss=triplet_loss, optimizer=optimizer)
    return model

In [ ]:
# Modify the prepare_dataset function to handle unpacked arguments
def prepare_dataset(anchor, positive, negative):
    inputs = {'input_1': anchor, 'input_2': positive, 'input_3': negative}
    labels = tf.zeros_like(anchor)[:, 0, 0]  # Dummy labels
    return inputs, labels

In [ ]:
# Create datasets
train_dataset = create_triplet_dataset(train_path)
val_dataset = create_triplet_dataset(val_path)

In [ ]:
# Prepare the datasets
batch_size = 16  # 2. Reduced batch size
epochs = 50

In [ ]:
# Create and compile the model
input_shape = (224, 224, 3)
model = create_model(input_shape)

In [ ]:
# Define optimizer globally
optimizer = Adam(learning_rate=0.0001)
optimizer = tf.keras.mixed_precision.LossScaleOptimizer(optimizer)

In [ ]:
train_dataset = (train_dataset
                 .cache()
                 .shuffle(1000)
                 .batch(batch_size)
                 .map(prepare_dataset, num_parallel_calls=tf.data.AUTOTUNE)
                 .prefetch(tf.data.AUTOTUNE))

In [ ]:
val_dataset = (val_dataset
               .cache()
               .batch(batch_size)
               .map(prepare_dataset, num_parallel_calls=tf.data.AUTOTUNE)
               .prefetch(tf.data.AUTOTUNE))

In [ ]:
# 4. Use a custom training loop for more control
@tf.function
def train_step(inputs, labels):
    with tf.GradientTape() as tape:
        predictions = model(inputs, training=True)
        loss = triplet_loss(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

In [ ]:
# Check dataset cardinality
train_cardinality = tf.data.experimental.cardinality(train_dataset).numpy()
val_cardinality = tf.data.experimental.cardinality(val_dataset).numpy()

In [ ]:
for epoch in range(epochs):
    print(f"\nEpoch {epoch+1}/{epochs}")

    # Training
    train_loss = tf.keras.metrics.Mean()
    for step, (inputs, labels) in enumerate(train_dataset):
        loss = train_step(inputs, labels)
        train_loss.update_state(loss)

        if step % 10 == 0:
            print(f"Step {step}, Loss: {train_loss.result():.4f}")

        # Clear memory periodically
        if step % 100 == 0:
            tf.keras.backend.clear_session()

        # Break if dataset is infinite (optional)
        if train_cardinality < 0 and step >= 1000:  # Adjust this number as needed
            break

    print(f"Training Loss: {train_loss.result():.4f}")

    # Validation
    val_loss = tf.keras.metrics.Mean()
    for inputs, labels in val_dataset:
        val_predictions = model(inputs, training=False)
        val_batch_loss = triplet_loss(labels, val_predictions)
        val_loss.update_state(val_batch_loss)

        # Break if dataset is infinite (optional)
        if val_cardinality < 0 and val_loss.count.numpy() >= 100:  # Adjust this number as needed
            break

    print(f"Validation Loss: {val_loss.result():.4f}")


Epoch 1/50
Step 0, Loss: 0.0000


In [ ]:
# Save the trained model
model.save('face_recognition_model.keras')

In [ ]:
# Save the base network for generating embeddings
base_network = create_base_network(input_shape)
base_network.set_weights(model.get_layer(base_network.name).get_weights())
base_network.save('face_embedding_model.keras')

In [ ]:
print("Training completed. Models saved.")